<a href="https://colab.research.google.com/github/guptagundlapalli/Applied_Data_Analytics/blob/master/SqueezeLLM_Make_Accurate_Quantized_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to quantize Llama 2 with SqueezeLLM to 3-bit and 4-bit.

You can follow the same steps for Mistral 7B, but set "mistral" for "--model_type".


First, install SqueezeLLM (you need a GPU):

In [ ]:
!git clone https://github.com/SqueezeAILab/SqueezeLLM
!cd SqueezeLLM && pip install -e .
!cd SqueezeLLM/squeezellm && python setup_cuda.py install

fatal: destination path 'SqueezeLLM' already exists and is not an empty directory.
Obtaining file:///content/SqueezeLLM
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for squeezellm (pyproject.toml) ... done
  Created wheel for squeezellm: filename=squeezellm-0.1.0-0.editable-py3-none-any.whl size=7572 sha256=bdaba6748c6e8c09add7dba921a476fa0250c7b649ec8740b72c72c68b444753
  Stored in directory: /tmp/pip-ephem-wheel-cache-xtnlmzzp/wheels/30/97/8c/2919c99f23ac8acc2921a339f74b1c532d5f2ad69abeca7ac8
Successfully built squeezellm
  Attempting uninstall: squeezellm
    Found existing installation: squeezellm 0.1.0
    Uninstalling squeezellm-0.1.0:
      Successfully uninstalled squeezellm-0.1.0
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutil

We will use Llama 2 from the Hugging Face. Make sure you have your Hugging Face access token registered locally or run the following cell.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

For computing the gradients used in the non-uniform quantization, we must also install the following:

In [ ]:
!git clone https://github.com/kssteven418/SqueezeLLM-gradients.git
!cd SqueezeLLM-gradients && pip install -e . && pip install -r requirements.txt

Cloning into 'SqueezeLLM-gradients'...
remote: Enumerating objects: 155017, done.
remote: Counting objects: 100% (499/499), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 155017 (delta 475), reused 477 (delta 465), pack-reused 154518
Receiving objects: 100% (155017/155017), 157.30 MiB | 35.86 MiB/s, done.
Resolving deltas: 100% (119044/119044), done.
Obtaining file:///content/SqueezeLLM-gradients
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-0.editable-py3-none-any.whl size=9737 sha256=165465a2fbd773deb20e855c44b82bd07c549efc3af3cd02bf461ef90659e7aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-2omnikno/wheels/b5/7a/38/ca28f74cf0120ac333362e3b573b9d12564b84792ea146acad
Suc

Here we compute the gradients. This cell needs a GPU. If you use Google Colab, only the A100 will have enough memory.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python SqueezeLLM-gradients/run.py --output_dir llama-gradients --model_name meta-llama/Llama-2-7b-hf  # single GPU

2024-02-08 04:25:08.290195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 04:25:08.290243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 04:25:08.291882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-08 04:25:09.412821: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Calibration with C4 
Resolving data files: 100% 1024/1024 [00:13<00:00, 77.95it/s]
Resolving data files: 100% 1024/1024 [00:01<00:00, 955.43it/s]
Resolving data files: 100% 7168/7168 [00:00<00:00, 387607.44it/s]
Resolving data files: 100% 64

Then, we chunk the model at layer level:

In [ ]:
!python SqueezeLLM/quantization/chunk_models.py --model meta-llama/Llama-2-7b-hf --output llama-chunks --model_type llama


chunking the model: meta-llama/Llama-2-7b-hf and storing in drive/MyDrive/llama-chunks
config.json: 100% 609/609 [00:00<00:00, 3.19MB/s]
model.safetensors.index.json: 100% 26.8k/26.8k [00:00<00:00, 89.0MB/s]
model-00001-of-00002.safetensors:   0% 0.00/9.98G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0% 31.5M/9.98G [00:00<00:35, 283MB/s]
model-00001-of-00002.safetensors:   1% 83.9M/9.98G [00:00<00:26, 369MB/s]
model-00001-of-00002.safetensors:   1% 136M/9.98G [00:00<00:24, 394MB/s] 
model-00001-of-00002.safetensors:   2% 189M/9.98G [00:00<00:23, 408MB/s]
model-00001-of-00002.safetensors:   2% 241M/9.98G [00:00<00:23, 416MB/s]
model-00001-of-00002.safetensors:   3% 294M/9.98G [00:00<00:23, 419MB/s]
model-00001-of-00002.safetensors:   3% 336M/9.98G [00:00<00:23, 418MB/s]
model-00001-of-00002.safetensors:   4% 388M/9.98G [00:00<00:22, 420MB/s]
model-00001-of-00002.safetensors:   4% 440M/9.98G [00:01<00:22, 422MB/s]
model-00001-of-00002.safetensors:   5% 493M/9.98G [00:01<00:22, 42

We do the same for the gradients computer earlier:

In [ ]:
!python SqueezeLLM/quantization/chunk_models.py --model llama-gradients --output llama-gradients-chunks --model_type llama

chunking the model: drive/MyDrive/llama-gradients and storing in drive/MyDrive/llama-gradients-chunks
Loading checkpoint shards: 100% 3/3 [02:47<00:00, 55.68s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at drive/MyDrive/llama-gradients and are newly initialized: ['model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.27

The following searches for the outliers to keep in the sparse matrix:

In [ ]:
!python SqueezeLLM/quantization/generate_outlier_config.py --model drive/MyDrive/llama-chunks --range 1.8 --output drive/MyDrive/llama-chunks-outlierconfig/

Model type : llama
  0% 0/32 [00:00<?, ?it/s]0 q % outlier: 7.062%
0 k % outlier: 5.751%
0 v % outlier: 2.658%
0 o % outlier: 3.800%
0 gate % outlier: 0.333%
0 up % outlier: 0.266%
0 down % outlier: 0.289%
  3% 1/32 [00:21<11:06, 21.49s/it]1 q % outlier: 6.485%
1 k % outlier: 5.458%
1 v % outlier: 2.533%
1 o % outlier: 4.695%
1 gate % outlier: 0.281%
1 up % outlier: 0.227%
1 down % outlier: 0.229%
  6% 2/32 [00:41<10:10, 20.36s/it]2 q % outlier: 2.780%
2 k % outlier: 3.912%
2 v % outlier: 0.324%
2 o % outlier: 0.332%
2 gate % outlier: 0.230%
2 up % outlier: 0.216%
2 down % outlier: 0.209%
  9% 3/32 [00:59<09:28, 19.62s/it]3 q % outlier: 1.796%
3 k % outlier: 2.264%
3 v % outlier: 0.372%
3 o % outlier: 0.366%
3 gate % outlier: 0.244%
3 up % outlier: 0.216%
3 down % outlier: 0.210%
 12% 4/32 [01:18<09:00, 19.32s/it]4 q % outlier: 1.377%
4 k % outlier: 1.795%
4 v % outlier: 0.291%
4 o % outlier: 0.269%
4 gate % outlier: 0.303%
4 up % outlier: 0.224%
4 down % outlier: 0.229%
 16% 5/32 [01:

# For 4-bit quantization

Now, we can perform the non-uniform quantization with kmeans. You only need a CPU for this step. If you use Google Colab, this will take several hours.

In [ ]:
!python SqueezeLLM/quantization/nuq.py --bit 4 --model_type llama --model drive/MyDrive/llama-chunks --gradient drive/MyDrive/llama-gradients-chunks --output drive/MyDrive/llama-LUT/ --outlier_config drive/MyDrive/llama-chunks-outlierconfig/outlier_config_o0.53.json --sensitivity 0.05

Running outlier mode with outlier folder threshold 0.53 and sensitivity 0.05
Quantizing layers [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Quantizing layer 0
Removing outliers outlier=0.53, sensitivity=0.05
removing outliers by sensitivity
p outlier: 0.04984307165590593
removing outliers by threshold
p outlier: 1.7908066665570352
100% 7/7 [09:58<00:00, 85.56s/it] 
Saving layer lut to drive/MyDrive/llama-LUT//lut/l0.pkl
Saving layer outliers to drive/MyDrive/llama-LUT//outliers/l0.pkl
Quantizing layer 1
Removing outliers outlier=0.53, sensitivity=0.05
removing outliers by sensitivity
p outlier: 0.049876672616276714
removing outliers by threshold
p outlier: 1.7513487623145543
100% 7/7 [09:40<00:00, 82.97s/it]
Saving layer lut to drive/MyDrive/llama-LUT//lut/l1.pkl
Saving layer outliers to drive/MyDrive/llama-LUT//outliers/l1.pkl
Quantizing layer 2
Removing outliers outlier=0.53, sensitivity=0.05
removing outliers 

Finally, we repack the model:

In [ ]:
!python SqueezeLLM/quantization/pack.py --model meta-llama/Llama-2-7b-hf --wbits 4 --folder drive/MyDrive/llama-LUT/ --save drive/MyDrive/llama-suqeezed --include_sparse --balance

config.json: 100% 609/609 [00:00<00:00, 2.20MB/s]
model.safetensors.index.json: 100% 26.8k/26.8k [00:00<00:00, 6.62MB/s]
model-00001-of-00002.safetensors:   0% 0.00/9.98G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0% 31.5M/9.98G [00:00<00:35, 284MB/s]
model-00001-of-00002.safetensors:   1% 73.4M/9.98G [00:00<00:29, 337MB/s]
model-00001-of-00002.safetensors:   1% 115M/9.98G [00:00<00:27, 357MB/s] 
model-00001-of-00002.safetensors:   2% 157M/9.98G [00:00<00:26, 366MB/s]
model-00001-of-00002.safetensors:   2% 199M/9.98G [00:00<00:26, 371MB/s]
model-00001-of-00002.safetensors:   2% 241M/9.98G [00:00<00:25, 375MB/s]
model-00001-of-00002.safetensors:   3% 283M/9.98G [00:00<00:25, 377MB/s]
model-00001-of-00002.safetensors:   3% 325M/9.98G [00:00<00:25, 383MB/s]
model-00001-of-00002.safetensors:   4% 367M/9.98G [00:00<00:25, 384MB/s]
model-00001-of-00002.safetensors:   4% 409M/9.98G [00:01<00:24, 388MB/s]
model-00001-of-00002.safetensors:   5% 451M/9.98G [00:01<00:24, 388MB/s]
model-0

# For 3-bit quantization

In [ ]:
!python SqueezeLLM/quantization/nuq.py --bit 3 --model_type llama --model drive/MyDrive/llama-chunks --gradient drive/MyDrive/llama-gradients-chunks --output drive/MyDrive/llama-LUT-3bit/ --outlier_config drive/MyDrive/llama-chunks-outlierconfig/outlier_config_o0.53.json --sensitivity 0.05

Running outlier mode with outlier folder threshold 0.53 and sensitivity 0.05
Quantizing layers [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Quantizing layer 0
Removing outliers outlier=0.53, sensitivity=0.05
removing outliers by sensitivity
p outlier: 0.04984307165590593
removing outliers by threshold
p outlier: 1.7908066665570352
100% 7/7 [05:10<00:00, 44.30s/it]
Saving layer lut to drive/MyDrive/llama-LUT-3bit//lut/l0.pkl
Saving layer outliers to drive/MyDrive/llama-LUT-3bit//outliers/l0.pkl
Quantizing layer 1
Removing outliers outlier=0.53, sensitivity=0.05
removing outliers by sensitivity
p outlier: 0.049876672616276714
removing outliers by threshold
p outlier: 1.7513487623145543
100% 7/7 [05:41<00:00, 48.72s/it]
Saving layer lut to drive/MyDrive/llama-LUT-3bit//lut/l1.pkl
Saving layer outliers to drive/MyDrive/llama-LUT-3bit//outliers/l1.pkl
Quantizing layer 2
Removing outliers outlier=0.53, sensitivity=0.05

In [ ]:
!python SqueezeLLM/quantization/pack.py --model meta-llama/Llama-2-7b-hf --wbits 3 --folder drive/MyDrive/llama-LUT-3bit/ --save drive/MyDrive/llama-squeezed-3bit --include_sparse --balance

Loading checkpoint shards: 100% 2/2 [00:26<00:00, 13.09s/it]
generation_config.json: 100% 188/188 [00:00<00:00, 756kB/s]
Running llama_sequential
Starting ...
llama_sequential Done: 202.52166318893433
Running llama_pack
Packing ...
model.layers.0.self_attn.k_proj
/content/SqueezeLLM/squeezellm/quant.py:126: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  outliers = outliers.to_sparse(layout=torch.sparse_csr)
self.numvals:  970119
self.rows:  4097
self.num_threads:  97024
nnz_per_thread:  10
model.layers.0.self_attn.o_proj
self.numvals:  645210
self.rows:  4097
self.num_threads:  64640
nnz_per_thread:  10
model.layers.0.self_attn.q_proj
self.numvals:  1189179
self.rows:  4097
self.num_threads:  119040
nnz_per_thread:  10
model.layers.0.self_attn.v_proj
self.numvals:  45425